# NonlinearModel
* Uses LightGBM to estimate a given (user, item) prediction

In [ ]:
import NBInclude: @nbinclude
@nbinclude("TreeModelBase.ipynb")

In [ ]:
train_model(
    [
        explicit_raw_alphas
        implicit_raw_alphas
        nondirectional_raw_alphas
        ["LinearExplicit", "LinearImplicit"]
    ],
    String[],
    ["implicit", "negative"],
    true,
    "validation",
    false,
    "NonlinearImplicit";
    λ = nothing,
);

In [ ]:
train_model(
    [
        explicit_raw_alphas
        implicit_raw_alphas
        ptw_raw_alphas
        nondirectional_raw_alphas
        ["LinearExplicit", "LinearImplicit", "LinearPtw"]
    ],
    String[],
    ["ptw", "negative"],
    true,
    "validation",
    false,
    "NonlinearPtw";
    λ = nothing,
);

In [ ]:
train_model(
    [
        explicit_raw_alphas
        implicit_raw_alphas
        nondirectional_raw_alphas
        ["LinearExplicit", "LinearImplicit"]
    ],
    ["LinearExplicit"],
    ["explicit"],
    false,
    "validation",
    false,
    "NonlinearExplicit";
    λ = nothing,
);

In [ ]:
function save_final_explicit_model()
    function model(split::String, content::String; raw_splits = true)
        @info "saving $split $content"
        split_fn = raw_splits ? get_raw_split : get_split
        alpha_fn = raw_splits ? read_raw_alpha : read_alpha
        if split == "training"
            return zeros(Float32, length(split_fn(split, content).user))
        end
        linear_split = alpha_fn("LinearExplicit", split, content).rating
        nonlinear_split = alpha_fn("NonlinearExplicit", split, content).rating
        linear_split + nonlinear_split
    end
    write_params(Dict("alphas" => ["LinearExplicit", "NonlinearExplicit"]), "Explicit")
    write_alpha(
        model,
        "Explicit";
        by_split = true,
        log = true,
        log_content = "explicit",
        log_splits = ["validation", "test"],
    )
end;

In [ ]:
save_final_explicit_model()